#### Implémenter Deep Q-Learning (DQN)

 DQN utilise un réseau de neurones pour approximer la Q-table, évitant ainsi de stocker toutes les valeurs possibles.

#### Imports

In [ ]:
import torch
import torch.nn as nn
# Importation du module torch.nn de PyTorch, qui contient des classes et des fonctions pour construire des réseaux de neurones
import torch.optim as optim
import random
import numpy as np
import gym

#### 📌 Étape 1 : Construire l'environnement

In [8]:
env = gym.make("Taxi-v3")
state_dim = env.observation_space.n   # Nombre d'états
action_dim = env.action_space.n       # Nombre d'actions

#### 📌 Étape 2 : Construire le Réseau de Neurones

In [ ]:
# Définition du réseau de neurones (DQN)
# Définition d'une classe DQN qui hérite de nn.Module : elle représente le réseau de neurones pour l'algorithme Deep Q-Learning
# 3 couches : une d’entrée (état), deux cachées, et une de sortie (actions possibles)
class DQN(nn.Module):
    # constructeur de la classe DQN,  Il prend en entrée la dimension de l'état et le nombre d'actions possibles.
    def __init__(self, input_dim, output_dim):
        super(DQN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)  # Première couche cachée et entièrement connectée, input_dim en entrée et 128 neurones en sortie
        self.fc2 = nn.Linear(128, 128)        # Deuxième couche cachée et entièrement connectée, 128 en entrée et 128 neurones en sortie
        self.fc3 = nn.Linear(128, output_dim) # Sortie = vecteur Q(s,a) avec une valeur pour chaque action / couche de sortie avec 128 neurones en entrée et output_dim neurones en sortie

    # Définition de la méthode forward qui spécifie le passage avant (forward pass) du réseau de neurones
    # Elle prend en entrée x, qui est l'état actuel
    def forward(self, x):
        x = torch.relu(self.fc1(x)) # On applique ReLU comme fonction d’activation
        x = torch.relu(self.fc2(x))
        return self.fc3(x)
    
# Ce réseau de neurones est utilisé dans l'algorithme Deep Q-Learning pour approximer la fonction de valeur d'action Q(s, a),
# qui est utilisée pour prendre des décisions optimales dans un environnement donné

#### 📌 Étape 3 : Entraîner le modèle

In [10]:
# Conversion en vecteur one-hot pour l'entrée du réseau
def one_hot_encoding(state, state_dim):
    state_vec = np.zeros(state_dim)
    state_vec[state] = 1
    return state_vec

# Initialisation du modèle DQN et de l'optimiseur
model = DQN(state_dim, action_dim)
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.MSELoss()  # Erreur quadratique moyenne

# Paramètres d'entraînement
gamma = 0.95
epsilon = 1.0
min_epsilon = 0.05
decay_rate = 0.005
n_episodes = 2000 # 20000
batch_size = 32
replay_memory = []

# Paramètres à ajouter : learning_rate, buffer_size, batch_size

In [11]:
for episode in range(n_episodes):
    state, _ = env.reset()
    state_vec = torch.tensor(one_hot_encoding(state, state_dim), dtype=torch.float32)
    done = False

    while not done:
        # Exploration vs Exploitation
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample()
        else:
            with torch.no_grad():
                q_values = model(state_vec)
                action = torch.argmax(q_values).item()

        # Effectuer l'action
        new_state, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated
        new_state_vec = torch.tensor(one_hot_encoding(new_state, state_dim), dtype=torch.float32)

        # Stocker l'expérience dans le buffer
        replay_memory.append((state_vec, action, reward, new_state_vec, done))
        if len(replay_memory) > 10000:  # Limite du buffer
            replay_memory.pop(0)

        # Échantillonner un mini-lot et entraîner le réseau
        if len(replay_memory) > batch_size:
            batch = random.sample(replay_memory, batch_size)
            states, actions, rewards, new_states, dones = zip(*batch)

            states = torch.stack(states)
            new_states = torch.stack(new_states)
            actions = torch.tensor(actions)
            rewards = torch.tensor(rewards, dtype=torch.float32)
            dones = torch.tensor(dones, dtype=torch.float32)

            q_values = model(states)
            next_q_values = model(new_states).detach()
            target_q_values = q_values.clone()

            for i in range(batch_size):
                target_q_values[i, actions[i]] = rewards[i] + (1 - dones[i]) * gamma * torch.max(next_q_values[i])

            loss = loss_fn(q_values, target_q_values)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # Mise à jour de l'état
        state_vec = new_state_vec

    # Diminuer epsilon pour moins d'exploration
    epsilon = max(min_epsilon, epsilon * np.exp(-decay_rate * episode))

c:\Users\fvenezia\AppData\Local\Programs\Python\Python312\Lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


#### 📌 Étape 4 : Évaluation

In [12]:
def evaluate_dqn(env, model, n_episodes=100):
    rewards = []
    for _ in range(n_episodes):
        state, _ = env.reset()
        state_vec = torch.tensor(one_hot_encoding(state, state_dim), dtype=torch.float32)
        total_reward = 0
        done = False

        while not done:
            with torch.no_grad():
                q_values = model(state_vec)
                action = torch.argmax(q_values).item()

            new_state, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated
            total_reward += reward
            state_vec = torch.tensor(one_hot_encoding(new_state, state_dim), dtype=torch.float32)

        rewards.append(total_reward)

    print(f"Récompense moyenne : {np.mean(rewards)}")

evaluate_dqn(env, model)

Récompense moyenne : -17.01
